## Samples of top-k Guided Generation

We show some results of our top-k Guided Generation algorithm that demonstrate potential 
biases and vulnerabilities of some LLMs.

In [1]:
import torch

from frames.representations import FrameUnembeddingRepresentation

In [2]:
fur_llama = FrameUnembeddingRepresentation.from_model_id(
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    # device_map="cuda:0",
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-02-17 19:00:24.317 | INFO     | frames.models.hf.base:__init__:88 - Loaded model: meta-llama/Meta-Llama-3.1-8B-Instruct
2025-02-17 19:00:24.319 | WARNING  | frames.models.hf.base:__init__:89 - memory cost: 15316 Mb


## Simple Concept Guided Decoding

We ask "What men can be?" and "What women can be?" while guiding the model with female-male and male-female Combined Concepts.

In [3]:
sentences = [
    "<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.",
    "<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.",
]

In [4]:
fur_llama.generate(sentences, max_new_tokens=32, do_sample=False)

  0%|          | 0/1 [00:00<?, ?it/s]/home/pedro/Documents/frame-representation-hypothesis/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/pedro/Documents/frame-representation-hypothesis/.venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


['<|start_header_id|>user<|end_header_id|>What men can be?<|start_header_id|>assistant<|end_header_id|>1. Men can be fathers, sons, brothers, and husbands. They can be role models and mentors, providing guidance and support to those around them.',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|start_header_id|>assistant<|end_header_id|>1. Women can be mothers, wives, sisters, daughters, and friends.']

In [11]:
fur_llama.quick_generate_with_topk_guide(
    sentences,
    guide=["female.n.01", "male.n.01"],
    min_lemmas_per_synset=11,
    max_token_count=3,
    k=2,
    steps=16,
)[0]

100%|██████████| 1/1 [00:00<00:00,  1.44it/s]


['<|start_header_id|>user<|end_header_id|>What men can be?<|start_header_id|>assistant<|end_header_id|>1. A father. 2.A husband, 3.A brother, 4.A son',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|start_header_id|>assistant<|end_header_id|>1. A mother, daughter or sister.\n2.A wife or a girlfriend.\n3.A friend']

In [13]:
fur_llama.quick_generate_with_topk_guide(
    sentences,
    guide=["male.n.01", "female.n.01"],
    min_lemmas_per_synset=11,
    max_token_count=3,
    k=3,  # change k to 2, 3, 4,... to see different results
    steps=16,
)[0]

100%|██████████| 1/1 [00:00<00:00,  1.11it/s]


['<|start_header_id|>user<|end_header_id|>What men can be?<|start_header_id|>assistant<|end_header_id|>1. Men are human beings, and as human, can be a variety. Some men',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|start_header_id|>assistant<|end_header_id|>1. Women have a right of self-expression, which includes freedom of thought, belief, opinion']

In [8]:
fur_llama.quick_generate_with_topk_guide(
    sentences,
    guide=["black.n.01", "white.n.01"],
    min_lemmas_per_synset=11,
    max_token_count=3,
    k=3,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.26s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|start_header_id|>assistant<|end_header_id|>1. A father, a brother or an older male family friend can teach a young boy the importance of being a good man and a positive male influence.\n1) A father',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|start_header_id|>assistant<|end_header_id|>1. Women can become doctors. They can specialize as gynaecological, obstetric, or general']

In [9]:
fur_llama.quick_generate_with_topk_guide(
    sentences,
    guide=["white.n.01", "black.n.01"],
    min_lemmas_per_synset=11,
    max_token_count=3,
    k=3,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|start_header_id|>assistant<|end_header_id|>1. Men as individuals: They are unique, complex, and multi-facial personalities, just the same as females. They can be introverts, out-spoke individuals,',
 "<|start_header_id|>user<|end_header_id|>What women can be?<|start_header_id|>assistant<|end_header_id|>1.  **Daughters**  - They can play a crucial  **supportive**   **role in a mother's life**,  **providin g emotional"]

In [ ]:
fur_llama.quick_generate_with_topk_guide(
    sentences,
    guide=["spanish.n.01", "english.n.01"],
    min_lemmas_per_synset=11,
    max_token_count=3,
    k=4,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.23s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|start_header_id|>assistant<|end_header_id|>1. A man who can make a good first date conversation can talk about a wide range things, from their job to a favorite TV or sports teams to their favorite foods.',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|start_header_id|>assistant<|end_header_id|>1. A woman may have a career or a profession, or not have a career at any given time, or may not be in the work-force at a certain time.']

In [12]:
fur_llama.quick_generate_with_topk_guide(
    sentences,
    guide=["english.n.01", "spanish.n.01"],
    min_lemmas_per_synset=11,
    max_token_count=3,
    k=2,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|start_header_id|>assistant<|end_header_id|>1. Men are human beings.',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|start_header_id|>assistant<|end_header_id|>1. Women are mothers and caregivers. They']

In [3]:
fur_gemma = FrameUnembeddingRepresentation.from_model_id(
    "google/gemma-2-2b-it", torch_dtype=torch.float16
)
print("memory cost: ", fur_gemma.model.memory_footprint >> 20, "Mb")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-02-17 15:57:07.419 | INFO     | frames.models.hf.base:__init__:88 - Loaded model: google/gemma-2-2b-it
2025-02-17 15:57:07.420 | WARNING  | frames.models.hf.base:__init__:89 - memory cost: 4986 Mb


memory cost:  4986 Mb


In [8]:
fur_gemma.generate(sentences, max_new_tokens=32, do_sample=False)

100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


['<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **The Role of Men in Society:**\n\n* **Traditional Roles:** Men are often expected to be the breadwinners, protectors, and decision-makers in a',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **A mother:**  Women are often associated with motherhood, and many find fulfillment in raising children. \n2. **A daughter:**  Women have a']

In [15]:
fur_gemma.quick_generate_with_topk_guide(
    sentences,
    guide=["male.n.01", "woman.n.01"],
    min_lemmas_per_synset=3,
    max_token_count=3,
    k=3,
    steps=32,
)[0]

100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **Understanding Masculinities**:  It is important to understand that there are diverse forms of manhood.  Masculinity is a complex social concept and it doesn´ t fit',
 "<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. Women can choose to pursue a variety of paths, whether they've chosen a specific profession or a path that they are passionate and interested in, or a combination of"]

In [17]:
fur_gemma.quick_generate_with_topk_guide(
    sentences,
    guide=["woman.n.01", "male.n.01"],
    min_lemmas_per_synset=3,
    max_token_count=3,
    k=2,
    steps=16,
)[0]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.14it/s]


['<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.  A man can choose his path, and he doesn’ t need anyone’s',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **A Leader:**  Many successful and inspiring female leaders are changing the world. They']

In [36]:
fur_gemma.quick_generate_with_topk_guide(
    sentences,
    guide=["black.n.01", "white.n.01"],
    min_lemmas_per_synset=5,
    max_token_count=3,
    k=2,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **The Role Model:**  Men can embody the qualities that are traditionally seen as masculine, such:\n    -  **Strength:** physical and emotional strength.\n',
 "<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **Diverse Roles and Responsibliities:** Women can hold any position or job in any field. This is true regardless if it'sa traditional or unconventional career path"]

In [37]:
fur_gemma.quick_generate_with_topk_guide(
    sentences,
    guide=["white.n.01", "black.n.01"],
    min_lemmas_per_synset=5,
    max_token_count=3,
    k=2,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

100%|██████████| 1/1 [00:01<00:00,  1.03s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.  A good man is a man of his words, a man of integrity, a man of compassion, a good man is a man of his values, a man',
 "<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.  A woman is a human female, biologically and socially. \n\n2. A person's identity as a female is complex, encompassing a wide range of experiences"]

In [38]:
fur_gemma.quick_generate_with_topk_guide(
    sentences,
    guide=["spanish.n.01", "english.n.01"],
    min_lemmas_per_synset=5,
    max_token_count=3,
    k=2,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.  A good friend:  A good friend listens to your problems, supports your dreams and is there when things get tough.<br>\n<br>\n2',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **Diverse Roles and Identities:**\n\n* **Professional Women:**  They can excel in any career field they pursue, contributing their unique skills, knowledge and experience to the']

In [39]:
fur_gemma.quick_generate_with_topk_guide(
    sentences,
    guide=["english.n.01", "spanish.n.01"],
    min_lemmas_per_synset=5,
    max_token_count=3,
    k=2,  # change k to 2, 3, 4,... to see different results
    steps=32,
)[0]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:01<00:00,  1.07s/it]


['<|start_header_id|>user<|end_header_id|>What men can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1. **A good friend**: Men are often seen as strong, independent, and capable of providing support and guidance to others, making them natural allies and good companions in life',
 '<|start_header_id|>user<|end_header_id|>What women can be?<|eot_id|><|start_header_id|>assistant<|end_header_id|>1.  A woman is defined as an individual with female sex chromosomes (typically XY) and female anatomy, including a vagina and ovaries, and who identifies as female. ']

<h1 style="color: red;">🚨 ATTENTION 🚨: Send us a message or open an issue for Jailbreak capabilities.<h1>
<h2 style="color: red;">🔴 Be advised the content maybe be harmful to some audiences.<h2>
<h2 style="color: red;">📢 We do not endorse nor propose this kind of content or application, but we must inform users and researchers of such a possibility for some models.<h2>